官方文件: [Merge, join, and concatenate](http://pandas.pydata.org/pandas-docs/stable/merging.html)

Outline:
* Merge on Key
    * Multiple keys
* Merge on Index
    * join
* Concatenate

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# Merge on Key

In [4]:
dframe1 = DataFrame({'key': list('XZYZXX'), 'data_set_1': np.arange(6)})
dframe1

,data_set_1,key
0,0,X
1,1,Z
2,2,Y
3,3,Z
4,4,X
5,5,X


In [15]:
dframe2 = DataFrame({'key': list('QYZ'), 'data_set_2': [1, 2, 3]})
dframe2

,data_set_2,key
0,1,Q
1,2,Y
2,3,Z


In [16]:
pd.merge(dframe1, dframe2)

,data_set_1,key,data_set_2
0,1,Z,3
1,3,Z,3
2,2,Y,2


In [17]:
pd.merge(dframe1, dframe2, on='key')

,data_set_1,key,data_set_2
0,1,Z,3
1,3,Z,3
2,2,Y,2


### how
參考[SQL join](http://stackoverflow.com/questions/38549/what-is-the-difference-between-inner-join-and-outer-join)

Default: how='inner'

In [18]:
# left
pd.merge(dframe1, dframe2, on='key', how='left')

,data_set_1,key,data_set_2
0,0,X,NaN
1,1,Z,3.0
2,2,Y,2.0
3,3,Z,3.0
4,4,X,NaN
5,5,X,NaN


In [19]:
# right
pd.merge(dframe1, dframe2, on='key', how='right')

,data_set_1,key,data_set_2
0,1.0,Z,3
1,3.0,Z,3
2,2.0,Y,2
3,NaN,Q,1


In [20]:
# outer
pd.merge(dframe1, dframe2, on='key', how='outer')

,data_set_1,key,data_set_2
0,0.0,X,NaN
1,4.0,X,NaN
2,5.0,X,NaN
3,1.0,Z,3.0
4,3.0,Z,3.0
5,2.0,Y,2.0
6,NaN,Q,1.0


In [22]:
dframe3 = DataFrame({'key': list('XXXYZZ'), 'data_set_3': range(6)})
dframe3

,data_set_3,key
0,0,X
1,1,X
2,2,X
3,3,Y
4,4,Z
5,5,Z


In [23]:
dframe4 = DataFrame({'key': list('YYXXZ'), 'data_set_4': range(5)})
dframe4

,data_set_4,key
0,0,Y
1,1,Y
2,2,X
3,3,X
4,4,Z


In [24]:
pd.merge(dframe3, dframe4)

,data_set_3,key,data_set_4
0,0,X,2
1,0,X,3
2,1,X,2
3,1,X,3
4,2,X,2
5,2,X,3
6,3,Y,0
7,3,Y,1
8,4,Z,4
9,5,Z,4


### Multiple keys

In [26]:
df_left = DataFrame({'key1': ['SF', 'SF', 'LA'],
                     'key2': ['one', 'two', 'one'],
                     'left_data': [10, 20, 30]})
df_left

,key1,key2,left_data
0,SF,one,10
1,SF,two,20
2,LA,one,30


In [27]:
df_right = DataFrame({'key1': ['SF', 'SF', 'LA', 'LA'],
                     'key2': ['one', 'one', 'one', 'two'],
                     'right_data': [40, 50, 60, 70]})
df_right

,key1,key2,right_data
0,SF,one,40
1,SF,one,50
2,LA,one,60
3,LA,two,70


In [33]:
pd.merge(df_left, df_right, on=['key1', 'key2'], how='outer')

,key1,key2,left_data,right_data
0,SF,one,10.0,40.0
1,SF,one,10.0,50.0
2,SF,two,20.0,NaN
3,LA,one,30.0,60.0
4,LA,two,NaN,70.0


In [31]:
pd.merge(df_left, df_right, on='key1')
# 若因為某些原因，merge的時候兩個data set都有的keys不在on之中，pandas會自動append _x / _y到這些keys

,key1,key2_x,left_data,key2_y,right_data
0,SF,one,10,one,40
1,SF,one,10,one,50
2,SF,two,20,one,40
3,SF,two,20,one,50
4,LA,one,30,one,60
5,LA,one,30,two,70


In [36]:
# 承上，改變自動append的name
pd.merge(df_left, df_right, on='key1', suffixes=('_lefty', '_righty'))

,key1,key2_lefty,left_data,key2_righty,right_data
0,SF,one,10,one,40
1,SF,one,10,one,50
2,SF,two,20,one,40
3,SF,two,20,one,50
4,LA,one,30,one,60
5,LA,one,30,two,70


# Merge on Index

In [38]:
df_left = DataFrame({'key': list('XYZXY'),
                     'data': range(5)})
df_left

,data,key
0,0,X
1,1,Y
2,2,Z
3,3,X
4,4,Y


In [41]:
df_right = DataFrame({'group_data': [10, 20]}, index=['X', 'Y'])
df_right

,group_data
X,10
Y,20


In [42]:
# Use the key for the left Dframe, and the index for the right
pd.merge(df_left, df_right, left_on='key', right_index=True)

,data,key,group_data
0,0,X,10
3,3,X,10
1,1,Y,20
4,4,Y,20


In [52]:
# 等同於
df_left.join(df_right, on='key', how='inner')
# join預設就是使用index，所以要讓df_left使用key而非index要指定on=
# join預設為left，所以這邊指定how='inner'

,data,key,group_data
0,0,X,10
3,3,X,10
1,1,Y,20
4,4,Y,20


### Hierarchal index

In [47]:
df_left_hr = DataFrame({'key1': ['SF','SF','SF','LA','LA'],
                        'key2': [10, 20, 30, 20, 30],
                        'data_set': np.arange(5)})
df_left_hr

,data_set,key1,key2
0,0,SF,10
1,1,SF,20
2,2,SF,30
3,3,LA,20
4,4,LA,30


In [45]:
df_right_hr = DataFrame(np.arange(10).reshape((5, 2)),
                        index=[['LA','LA','SF','SF','SF'], [20, 10, 10, 10, 20]], # Hierarchal index
                        columns=['col_1', 'col_2'])
df_right_hr

col_1  col_2
LA 20      0      1
   10      2      3
SF 10      4      5
   10      6      7
   20      8      9

In [48]:
pd.merge(df_left_hr, df_right_hr, left_on=['key1', 'key2'], right_index=True)

,data_set,key1,key2,col_1,col_2
0,0,SF,10,4,5
0,0,SF,10,6,7
1,1,SF,20,8,9
3,3,LA,20,0,1


# Concatenate

### numpy - ndarray

In [53]:
arr1 = np.arange(9).reshape(3, 3)
arr1

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [57]:
np.concatenate([arr1, arr1]) # Default: cross index(axis=0)

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8],
       [0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [56]:
np.concatenate([arr1, arr1], axis=1)

array([[0, 1, 2, 0, 1, 2],
       [3, 4, 5, 3, 4, 5],
       [6, 7, 8, 6, 7, 8]])

### pandas  - Series

In [59]:
ser1 = Series([0, 1, 2], index=['T', 'U','V'])
ser1

T    0
U    1
V    2
dtype: int64

In [60]:
ser2 = Series([3, 4], index=['X', 'Y'])
ser2

X    3
Y    4
dtype: int64

In [61]:
pd.concat([ser1, ser2])

T    0
U    1
V    2
X    3
Y    4
dtype: int64

In [62]:
# DataFrame
pd.concat([ser1, ser2], axis=1)

,0,1
T,0.0,NaN
U,1.0,NaN
V,2.0,NaN
X,NaN,3.0
Y,NaN,4.0


In [63]:
# Hierarchal index
pd.concat([ser1, ser2], keys=['cat1', 'cat2'])

cat1  T    0
      U    1
      V    2
cat2  X    3
      Y    4
dtype: int64

### pandas - DataFrame

In [64]:
dframe1 = DataFrame(np.random.randn(4, 3), columns=list('XYZ'))
dframe1

,X,Y,Z
0,-0.527021,0.106045,-1.947517
1,1.218732,-0.672439,0.288069
2,0.526919,0.281322,0.515197
3,-0.061393,-0.357474,0.476145


In [65]:
dframe2 = DataFrame(np.random.randn(3, 3), columns=list('YQX'))
dframe2

,Y,Q,X
0,-0.960690,1.289036,-1.272896
1,1.382131,-0.604738,-0.788984
2,-0.827140,-0.750249,-1.949914


In [80]:
f = pd.concat([dframe1, dframe2])
f
# f.ix[1]

,Q,X,Y,Z
0,NaN,-0.527021,0.106045,-1.947517
1,NaN,1.218732,-0.672439,0.288069
2,NaN,0.526919,0.281322,0.515197
3,NaN,-0.061393,-0.357474,0.476145
0,1.289036,-1.272896,-0.960690,NaN
1,-0.604738,-0.788984,1.382131,NaN
2,-0.750249,-1.949914,-0.827140,NaN


In [71]:
pd.concat([dframe1, dframe2], ignore_index=True)

,Q,X,Y,Z
0,NaN,-0.527021,0.106045,-1.947517
1,NaN,1.218732,-0.672439,0.288069
2,NaN,0.526919,0.281322,0.515197
3,NaN,-0.061393,-0.357474,0.476145
4,1.289036,-1.272896,-0.960690,NaN
5,-0.604738,-0.788984,1.382131,NaN
6,-0.750249,-1.949914,-0.827140,NaN


# Combine DataFrames

### Series

In [84]:
ser1 = Series([2, np.nan, 4, np.nan, 6, np.nan],
              index=list('QRSTUV'))
ser1

Q    2.0
R    NaN
S    4.0
T    NaN
U    6.0
V    NaN
dtype: float64

In [87]:
ser2 = Series(np.arange(len(ser1)),
              dtype=np.float64,
              index=list('QRSTUV'))
ser2

Q    0.0
R    1.0
S    2.0
T    3.0
U    4.0
V    5.0
dtype: float64

In [90]:
Series(np.where(pd.isnull(ser1), ser2, ser1), index=ser1.index)

Q    2.0
R    1.0
S    4.0
T    3.0
U    6.0
V    5.0
dtype: float64

In [96]:
# 等同
ser1.combine_first(ser2) # combine_first: 以ser1的value為主，若沒有才用ser2的

Q    2.0
R    1.0
S    4.0
T    3.0
U    6.0
V    5.0
dtype: float64

### DataFrame

In [93]:
dframe_odds = DataFrame({'X': [1., np.nan, 3., np.nan],
                         'Y': [np.nan, 5., np.nan, 7.],
                         'Z': [np.nan, 9., np.nan, 11.]})
dframe_odds

,X,Y,Z
0,1.0,NaN,NaN
1,NaN,5.0,9.0
2,3.0,NaN,NaN
3,NaN,7.0,11.0


In [94]:
dframe_evens = DataFrame({'X': [2., 4., np.nan, 6., 8.],
                          'Y': [np.nan, 10., 12., 14., 16.]})
dframe_evens

,X,Y
0,2.0,NaN
1,4.0,10.0
2,NaN,12.0
3,6.0,14.0
4,8.0,16.0


In [95]:
dframe_odds.combine_first(dframe_evens)

,X,Y,Z
0,1.0,NaN,NaN
1,4.0,5.0,9.0
2,3.0,12.0,NaN
3,6.0,7.0,11.0
4,8.0,16.0,NaN
